In [1]:
from requests import get
import pandas as pd
from bs4 import BeautifulSoup
from IPython.core.display import clear_output
from time import time
from time import sleep
from random import randint

names_female=[]
names_male = []

gender = ['femininos', 'masculinos']

for g in gender:
    if g == 'femininos':
        aux = 152
    else: 
        aux = 194
    pages = [str(i) for i in range(1,aux+1)]

    start_time = time()
    requests = 0

    for page in pages:
        # Make a get request
        url = 'https://www.dicionariodenomesproprios.com.br/nomes-'+ g +'/{}/'.format(page)
        response = get(url)

        # Pause the loop
        sleep(randint(8,15))

        # Monitor the requests
        requests += 1
        elapsed_time = time() - start_time
        print('Request:{}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
        clear_output(wait = True)

        # Throw a warning for non-200 status codes
        if response.status_code != 200:
            warn('Request: {}; Status code: {}'.format(requests, response.status_code))
  
        # Break the loop if the number of requests is greater than expected
        if requests > len(pages):
            warn('Number of requests was greater than expected.')  
            break 

        # using BeautifulSoup
        html_soup = BeautifulSoup(response.text, 'html.parser')

        # nomes_containers = html_soup.find_all('ul', class_ = 'names-list')
        nomes_containers = html_soup.find_all('div', class_ = 'col-xs-12 lista')
        first=nomes_containers[0]

        aa = first.find_all('li')

        for li in aa:
            bb = li.a.text
            if g == 'femininos': 
                names_female.append(bb)
            else:
                names_male.append(bb)

Request:194; Frequency: 0.07931270601193811 requests/s


In [2]:
names_dictionary = pd.concat([pd.DataFrame(names_male), pd.DataFrame(names_female)], ignore_index=True, axis=1)
names_dictionary = names_dictionary.apply(lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('latin-1'))
names_dictionary = names_dictionary.apply(lambda x: x.astype(str).str.lower())
names_dictionary.columns = ['Male', 'Female']

male = names_dictionary['Male'].dropna(axis=0)
female = names_dictionary['Female'].dropna(axis=0)

m1 = male.str.split().str[0].dropna(axis=0)
m2 = male.str.split().str[1].dropna(axis=0)
m3 = m1.append(m2).drop_duplicates()

f1 = female.str.split().str[0].dropna(axis=0)
f2 = female.str.split().str[1].dropna(axis=0)
f3 = f1.append(f2).drop_duplicates()

m = m3.to_frame().reset_index(drop=True)
f = f3.to_frame().reset_index(drop=True)

names_dictionary = pd.concat([m,f], ignore_index=True, axis=1).reset_index(drop=True)
names_dictionary.columns = ['Male', 'Female']

names_dictionary.to_csv("names_dictionary.csv")